In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201129.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice]
0,11/24/2020 22:06:25,The Blob (1958),Them! (1954),Grave of the Fireflies (1988),The Host (2006),Taxidermia (2006),Force Majeure (2014),Spiderman: Into the Spiderverse (2018),NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,SORRY I AM VERY BAD AT MY JOB,DOLLY PARTON,GOOD VIBES ONLY,A MUSICAL (NOT CATS),A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,[Default to Nicolas Cage],COMEDY OF MANNERS
1,11/24/2020 22:16:04,Them! (1954),Grave of the Fireflies (1988),Taxidermia (2006),The Blob (1958),NaN,NaN,NaN,NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,SORRY I AM VERY BAD AT MY JOB,COMEDY OF MANNERS,A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,GOOD VIBES ONLY,A MUSICAL (NOT CATS),[Default to Nicolas Cage],DOLLY PARTON
2,11/24/2020 23:17:11,Force Majeure (2014),Grave of the Fireflies (1988),The Blob (1958),The Host (2006),Taxidermia (2006),Them! (1954),Spiderman: Into the Spiderverse (2018),NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,SORRY I AM VERY BAD AT MY JOB,GOOD VIBES ONLY,A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,DOLLY PARTON,COMEDY OF MANNERS,[Default to Nicolas Cage],A MUSICAL (NOT CATS)
3,11/24/2020 23:26:42,Spiderman: Into the Spiderverse (2018),Them! (1954),The Host (2006),The Blob (1958),Force Majeure (2014),Grave of the Fireflies (1988),Taxidermia (2006),SORRY I AM VERY BAD AT MY JOB,[Default to Nicolas Cage],A MUSICAL (NOT CATS),COMEDY OF MANNERS,DOLLY PARTON,A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,GOOD VIBES ONLY
4,11/25/2020 5:37:13,The Host (2006),Them! (1954),Grave of the Fireflies (1988),Spiderman: Into the Spiderverse (2018),Force Majeure (2014),The Blob (1958),NaN,COMEDY OF MANNERS,NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,GOOD VIBES ONLY,SORRY I AM VERY BAD AT MY JOB,[Default to Nicolas Cage],NaN,NaN
5,11/25/2020 7:04:10,Grave of the Fireflies (1988),Force Majeure (2014),The Host (2006),The Blob (1958),Them! (1954),Spiderman: Into the Spiderverse (2018),Taxidermia (2006),NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,COMEDY OF MANNERS,SORRY I AM VERY BAD AT MY JOB,A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,GOOD VIBES ONLY,DOLLY PARTON,A MUSICAL (NOT CATS),[Default to Nicolas Cage]
6,11/25/2020 7:51:55,Force Majeure (2014),Taxidermia (2006),The Host (2006),The Blob (1958),Grave of the Fireflies (1988),Them! (1954),Spiderman: Into the Spiderverse (2018),A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,SORRY I AM VERY BAD AT MY JOB,COMEDY OF MANNERS,A MUSICAL (NOT CATS),NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,GOOD VIBES ONLY,[Default to Nicolas Cage],DOLLY PARTON
7,11/25/2020 17:15:45,The Blob (1958),Grave of the Fireflies (1988),The Host (2006),Spiderman: Into the Spiderverse (2018),Taxidermia (2006),Force Majeure (2014),Them! (1954),SORRY I AM VERY BAD AT MY JOB,GOOD VIBES ONLY,A FINE CHEESE: MOVIES THAT RIPEN WITH AGE,A MUSICAL (NOT CATS),NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T D...,COMEDY OF MANNERS,DOLLY PARTON,[Defau

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['The Blob (1958)', 'Them! (1954)', 'Grave of the Fireflies (1988)',
       'The Host (2006)', 'Taxidermia (2006)', 'Force Majeure (2014)',
       'Spiderman: Into the Spiderverse (2018)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'The Blob (1958)': <Candidate('The Blob (1958)')>,
 'Them! (1954)': <Candidate('Them! (1954)')>,
 'Grave of the Fireflies (1988)': <Candidate('Grave of the Fireflies (1988)')>,
 'The Host (2006)': <Candidate('The Host (2006)')>,
 'Taxidermia (2006)': <Candidate('Taxidermia (2006)')>,
 'Force Majeure (2014)': <Candidate('Force Majeure (2014)')>,
 'Spiderman: Into the Spiderverse (2018)': <Candidate('Spiderman: Into the Spiderverse (2018)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The Blob (1958), Them! (1954), Grave of the Fireflies (1988), The Host (2006), Taxidermia (2006), Force Majeure (2014), Spiderman: Into the Spiderverse (2018))>,
 <Ballot(Them! (1954), Grave of the Fireflies (1988), Taxidermia (2006), The Blob (1958))>,
 <Ballot(Force Majeure (2014), Grave of the Fireflies (1988), The Blob (1958), The Host (2006), Taxidermia (2006), Them! (1954), Spiderman: Into the Spiderverse (2018))>,
 <Ballot(Spiderman: Into the Spiderverse (2018), Them! (1954), The Host (2006), The Blob (1958), Force Majeure (2014), Grave of the Fireflies (1988), Taxidermia (2006))>,
 <Ballot(The Host (2006), Them! (1954), Grave of the Fireflies (1988), Spiderman: Into the Spiderverse (2018), Force Majeure (2014), The Blob (1958))>,
 <Ballot(Grave of the Fireflies (1988), Force Majeure (2014), The Host (2006), The Blob (1958), Them! (1954), Spiderman: Into the Spiderverse (2018), Taxidermia (2006))>,
 <Ballot(Force Majeure (2014), Taxidermia (2006), The Host (2006), The B

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                 Votes  Status
--------------------------------------  -------  --------
Force Majeure (2014)                          3  Hopeful
The Blob (1958)                               2  Hopeful
Grave of the Fireflies (1988)                 1  Hopeful
Them! (1954)                                  1  Hopeful
The Host (2006)                               1  Hopeful
Spiderman: Into the Spiderverse (2018)        1  Rejected
Taxidermia (2006)                             0  Rejected

ROUND 2
Candidate                                 Votes  Status
--------------------------------------  -------  --------
Force Majeure (2014)                          3  Hopeful
Them! (1954)                                  2  Hopeful
The Blob (1958)                               2  Hopeful
Grave of the Fireflies (1988)                 1  Rejected
The Host (2006)                               1  Rejected
Spiderman: Into the Spiderverse (2018)        0  Rejected
Taxidermi

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                                Votes  Status
-----------------------------------------------------  -------  --------
NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T DIE HARD        4  Hopeful
SORRY I AM VERY BAD AT MY JOB                                2  Hopeful
A FINE CHEESE: MOVIES THAT RIPEN WITH AGE                    1  Hopeful
COMEDY OF MANNERS                                            1  Hopeful
DOLLY PARTON                                                 1  Rejected
GOOD VIBES ONLY                                              0  Rejected
[Default to Nicolas Cage]                                    0  Rejected
A MUSICAL (NOT CATS)                                         0  Rejected

ROUND 2
Candidate                                                Votes  Status
-----------------------------------------------------  -------  --------
NON-TRADITIONAL CHRISTMAS MOVIES THAT AREN'T DIE HARD        4  Hopeful
SORRY I AM VERY BAD AT MY JOB              